$s$

In [6]:
import ipywidgets as widgets
out = widgets.Output()
with out:
    
    %bookmark 'model_dir'
    current_dir = %pwd
    while 'models' in current_dir:
        %cd ..
        current_dir = %pwd
    %bookmark 'base_dir'
    %cd source_code
    out.clear_output()
%run base
%run gui_v6
with out:
    %cd ..
    %cd models/fourbar
    out.clear_output()


In [7]:
m=model()
m.show()

## VEHICLE DYNAMICS MODELING AND SIMULATION TOOL

In [8]:
m.topology.edges

OutMultiEdgeView([(rbs_ground, rbs_link, 'jcs_rev')])

In [21]:
def equations_assembler(topology):
    
    edgelist = topology.edges(data='obj')
    nodelist = topology.nodes
    
    n_nodes = len(nodelist)
    n_edges = len(edgelist)
            
    jacobian = np.zeros((n_edges+n_nodes,n_nodes),dtype=np.object)
    jacobian.fill(None)
    
    equations = np.zeros((n_edges+n_nodes),dtype=np.object)
    
    vel_rhs = np.zeros((n_edges+n_nodes),dtype=np.object)
    acc_rhs = np.zeros((n_edges+n_nodes),dtype=np.object)
    
    reaction_vector = np.zeros((n_edges),dtype=np.object)
    reaction_indecies = np.zeros((n_edges),dtype=np.object)
    
    lamda_vector = np.zeros((n_edges+n_nodes),dtype=np.object)
    
    node_index = dict( (node,i) for i,node in enumerate(nodelist) )
    
    n7 = np.arange(7)
    for ei,e in enumerate(edgelist):
        (u,v) = e[:2]
        eo    = e[2]
        
        ui = node_index[u]
        vi = node_index[v]
        
        ui7 = (ui*7)+n7
        vi7 = (vi*7)+n7
        
        if jacobian[ui+n_edges,ui]==None: jacobian[ui+n_edges,ui] = (u.jac,ui7)
        if jacobian[vi+n_edges,vi]==None: jacobian[vi+n_edges,vi] = (v.jac,vi7)
                
        jacobian[ei,ui] = (eo.jacobian_i,ui7,vi7)
        jacobian[ei,vi] = (eo.jacobian_j,ui7,vi7)
        
        equations[ei] = (eo.equations,ui7,vi7)
        if equations[ui+n_edges]==0: equations[ui+n_edges]=(u.equations,ui7)
        if equations[vi+n_edges]==0: equations[vi+n_edges]=(v.equations,vi7)
        
        
        vel_rhs[ei] = ((eo.vel_rhs) if eo.nc==1 else np.zeros((eo.nc,1)))
        vel_rhs[ui+n_edges] = (np.array([[0]]) if u.nc==1 else np.zeros((7,1)))
        vel_rhs[vi+n_edges] = (np.array([[0]]) if v.nc==1 else np.zeros((7,1)))
        
        acc_rhs[ei] = (eo.acc_rhs,ui7,vi7)
        if acc_rhs[ui+n_edges]==0: acc_rhs[ui+n_edges]=(u.acc_rhs,ui7)
        if acc_rhs[vi+n_edges]==0: acc_rhs[vi+n_edges]=(v.acc_rhs,vi7)
            
        reaction_vector[ei] = (eo.reactions,ui7,vi7,eo.index)
        reaction_indecies[ei] = eo.reaction_index
        
        lamda_vector[ei] = eo.index
        if lamda_vector[ui+n_edges]==0: lamda_vector[ui+n_edges]=u.index
        if lamda_vector[vi+n_edges]==0: lamda_vector[vi+n_edges]=v.index
        
    #####################################################
    # Defining vectorized functions
    #####################################################
    
    def j_mapper(i,q):
        if len(i)==3:
            fun,bi,bj = i
            return fun(q[bi],q[bj])
        else:
            fun,bi = i
            return fun(q[bi])
    j_vectorized = np.vectorize(j_mapper,otypes=[np.object],excluded='q')
    
    
    def v_mapper(i):
        if isinstance(i,(int,np.ndarray)):
            return i
        else:
            return i()
    v_vectorized = np.vectorize(v_mapper,otypes=[np.object])
    
    def a_mapper(i,q,qd):
        if len(i)==3:
            fun,bi,bj = i
            return fun(q[bi],q[bj],qd[bi],qd[bj])
        else:
            fun,bi = i
            return fun(qd[bi])
    a_vectorized = np.vectorize(a_mapper,otypes=[np.object],excluded=('q','qd'))
    
    def r_mapper(i,q,lamda):
        fun,bi,bj,index = i
        return fun(q[bi],q[bj],lamda[index].values)
    r_vectorized = np.vectorize(r_mapper,otypes=[np.object],excluded=('q','lamda'))
    
    ####################################################################################
    ####################################################################################
    
    mass_matrix = np.zeros((n_nodes,n_nodes),dtype=np.object)
    mass_matrix.fill(None)
    
    gravitational_vector = np.zeros((n_nodes),dtype=np.object)
    centrifugal_vector   = np.zeros((n_nodes),dtype=np.object)
    inertiaforce_vector  = np.zeros((n_nodes),dtype=np.object)
    
    for n in nodelist:
        ni = node_index[n]
        ni7 = (ni*7)+n7
        
        mass_matrix[ni,ni]=(n.mass_matrix,ni7)
        
        gravitational_vector[ni]=n.gravity
        
        centrifugal_vector[ni]=(n.centrifugal,ni7)
        
        inertiaforce_vector[ni]=(n.inertia_force,ni7)
        
        
    def mass_mapper(i,q):
        fun,bi = i
        return fun(q[bi])
    mass_vectorized = np.vectorize(mass_mapper,otypes=[np.object],excluded='q')
    
    def centrifugal_mapper(i,q,qd):
        fun,bi = i
        return fun(q[bi],qd[bi])
    centrifugal_vectorized = np.vectorize(centrifugal_mapper,otypes=[np.object],excluded=('q','qd'))
    
    def gravity_mapper(i):
        return i()
    gravity_vectorized = np.vectorize(gravity_mapper,otypes=[np.object])
    
    
    system = pd.Series([jacobian,
                        jacobian.nonzero(),
                        equations, 
                        vel_rhs, 
                        acc_rhs,
                        reaction_vector,
                        reaction_indecies,
                        lamda_vector,
                        mass_matrix,
                        gravitational_vector,
                        centrifugal_vector,
                        inertiaforce_vector,
                        j_vectorized,
                        v_vectorized,
                        a_vectorized,
                        r_vectorized,
                        mass_vectorized,
                        centrifugal_vectorized,
                        gravity_vectorized],
                       index=['Jacobian','Jnzi','Ceq','Veq','Aeq','Rvec','Rind','Lind',
                              'MassMatrix','Gravity','Centrifugal','Inertia',
                              'Jf','Vf','Af','Rf','Mf','Cf','Gf'])
    
    return system


In [12]:
def jacobian_evaluator(jac_blocks,nzi,mapper,q):
    A = jac_blocks.copy()
    A[nzi]=mapper(A[nzi],q=q)
    return sc.sparse.bmat(A,format='csc')

In [50]:
def coordinates_mapper(q):
    n=len(q)
    int2str=pd.Series(q.index,index=np.arange(0,n,1))
    str2int=pd.Series(np.arange(0,n,1),index=q.index)
    return int2str, str2int

def extract_ind(sparse_jac,q):
    mat=sparse_jac.A
    rows,cols=mat.shape
    permR=sc.linalg.lu(mat.T)[0]
    ind_cols=permR[:,rows:]
    maped=coordinates_mapper(q)[0]
    ind_coord=[maped[np.argmax(ind_cols[:,i])] for i in range(cols-rows) ]
    
    return ind_coord, ind_cols, permR


def dds(q0,qd0,system_equations,time_array):
    '''
    Dynamically Driven Systems Solver
    '''
    #Qa_f = system_equations.Qa
    Cf = system_equations.Cf
    Gf = system_equations.Gf
    Mf  = system_equations.Mf
    Jf = eq_f = system_equations.Jf
    Af = system_equations.Af
    Vf = system_equations.Vf
    
    Cv = system_equations.Centrifugal
    Gv = system_equations.Gravity
    Mm  = system_equations.MassMatrix
    Jm = eq_f = system_equations.Jacobian
    Av = system_equations.Aeq
    Vv = system_equations.Veq
    Ev = system_equations.Ceq
    Jnzi = system_equations.Jnzi

    # assigning initial conditions to the system NE equations
    M  = sc.sparse.block_diag(Mf(Mm.diagonal(),q=q0.values),format='csc')
    J  = jacobian_evaluator(Jm,Jnzi,Jf,q0.values)
    Qv = np.concatenate(Cf(Cv,q=q0.values,qd=qd0.values))
    Qg = np.concatenate(Gf(Gv))
    Qt = (Qv+Qg)
    ac = np.concatenate(Af(Av,q=q0.values,qd=qd0.values))
    
    # Initiating coordinate partitioning
    qind=extract_ind(J,q0)
    qstr=qind[0]
    Ids=qind[1]
    perm_matrix = qind[2]
    qind_index=list(coordinates_mapper(q0)[1][qstr])
    print('DOF : %s \n' %len(qstr))
    print('Independent Coordinates are: %s with indices: %s \n'%(qstr,qind_index))
    
    
    # creating dataframes to hold the simulation results at each timestep
    # with initial conditions at t0
    position_df=pd.DataFrame(columns=q0.index)
    position_df.loc[0]=q0
    
    velocity_df=pd.DataFrame(columns=q0.index)
    velocity_df.loc[0]=qd0
    
    acceleration_df=pd.DataFrame(columns=q0.index)
    
    lamda_df=pd.DataFrame(columns=np.concatenate(system_equations.Lind))
    
    JR_df=pd.DataFrame(columns=np.concatenate(system_equations.Rind))
    


    # assembling the coefficient matrix and the rhs vector and solving for
    # system accelerations and lagrange multipliers
    coeff_matrix=sc.sparse.bmat([[M,Cq.T],[Cq,None]],format='csc')
    b_vector=np.concatenate([Qt,ac])
    x=sc.sparse.linalg.spsolve(coeff_matrix,b_vector)
    
    # updating the dataframes with the evaluated results
    qdd0n  = x[:7*nb] # the first 7xnb elements in the x vector
    lamda0 = x[7*nb:] # the rest of elements in the x vector
    acceleration_df.loc[0]=qdd0n
    lamda_df.loc[0]=lamda0
    reaction=JR_f(joints,q0,lamda_df.loc[0])
    JR_df.loc[0]=reaction.values.reshape((len(reaction,)))

    
    # Setting up the integrator function and the initial conditions
    ssm=state_space_creator(qind_index)
    r=ode(ssm).set_integrator('dop853')
    y0=assign_initial_conditions(q0,qd0,qstr)
    r.set_initial_value(y0).set_f_params(M,Cq,Qt,lamda0)
    
    # Setting up the time array to be used in integration steps and starting
    # the integration
    t=np.arange(0,sim_time,stepsize)
    for i,dt in enumerate(t):
#        print('time_step: '+str(i))
        progress_bar(len(t),i)

        r.integrate(dt)
#        print(r.y)
#        if not r.successful():
#            print("BREAKING SOLVER")
#            return position_df,velocity_df,acceleration_df,JR_df
        
        # creating the guess vector for the vector q as the values of the 
        # previous step and the value of newly evaluated independent coordinate
        guess=position_df.loc[i]
        guess[qstr]=r.y[:len(r.y)//2]
        
        # Evaluating the dependent vector q using newton raphson
        dependent=nr_dds(eq_f,Cq_f,guess,bodies,joints,actuators,Ids)
        position_df.loc[i+1]=dependent[0]
        Cq_new=dependent[1]
        # Calculating the system velocities
        vrhs=velf(actuators)
        vind=np.array(r.y[len(r.y)//2:]).reshape((len(r.y)//2,1))
        vrhs=np.concatenate([vrhs,vind])
        velocity_df.loc[i+1]=sc.sparse.linalg.spsolve(Cq_new,vrhs)
        
        
        q=position_df.loc[i+1]
        qd=velocity_df.loc[i+1]
        
        # Evaluating the new coeff matrix bloks of the system NE equations
        M  = M_f(q,bodies)
        Cq = Cq_f(q,bodies,joints,actuators)
        Qa = Qa_f(forces,q,qd)
        Qv = Qv_f(bodies,q,qd)
        Qg = Qg_f(bodies)
        Qt = (Qa+Qv+Qg).reshape((7*nb,))
        ac = accf(q,qd,bodies,joints,actuators)

        coeff_matrix=sc.sparse.bmat([[M,Cq.T],[Cq,None]],format='csc')
        b_vector=np.concatenate([Qt,ac])
        # Evaluating the acceleration and lagrange mult. vector
        x=sc.sparse.linalg.spsolve(coeff_matrix,b_vector)
        
        # updating the dataframes with the evaluated results
        qdd   = x[:7*nb] # the first 7xnb elements in the x vector
        lamda = x[7*nb:] # the rest of elements in the x vector
        acceleration_df.loc[i+1]=qdd
        lamda_df.loc[i+1]=lamda
#        spring_df.loc[i+1]=[spring.defflection,spring.velocity,spring.springforce,spring.damperforce]
        reaction=JR_f(joints,q,lamda_df.loc[i+1])
        JR_df.loc[i+1]=reaction.values.reshape((len(reaction,)))
        

        # Setting the ssm input parameters
        r.set_f_params(M,Cq_new[:M.shape[0]-len(qind_index),:],Qt,lamda)
    
    
    return position_df,velocity_df,acceleration_df,JR_df
    return M,J,Qt,ac


In [24]:
import scipy as sc
q0=pd.concat([i.q0 for i in m.topology.nodes])
assembled = equations_assembler(m.topology)
np.concatenate(assembled.Rind)

array(['jcs_rev_Fx', 'jcs_rev_Fy', 'jcs_rev_Fz', 'jcs_rev_Mx',
       'jcs_rev_My', 'jcs_rev_Mz'],
      dtype='<U10')

In [52]:
dds(q0,q0,assembled,0)

DOF : 3 

Independent Coordinates are: ['rbs_link.z', 'rbs_link.e0', 'rbs_link.y'] with indices: [9, 10, 8] 



(<14x14 sparse matrix of type '<class 'numpy.float64'>'
 	with 25 stored elements in Compressed Sparse Column format>,
 <11x14 sparse matrix of type '<class 'numpy.float64'>'
 	with 29 stored elements in Compressed Sparse Column format>,
 array([[  0.00000000e+00],
        [  0.00000000e+00],
        [ -9.81000000e+03],
        [  0.00000000e+00],
        [  0.00000000e+00],
        [  0.00000000e+00],
        [  0.00000000e+00],
        [  0.00000000e+00],
        [  0.00000000e+00],
        [ -1.06787918e+08],
        [  0.00000000e+00],
        [  0.00000000e+00],
        [  0.00000000e+00],
        [  0.00000000e+00]]),
 matrix([[ 2000.],
         [    0.],
         [    0.],
         [    0.],
         [    0.],
         [    0.],
         [    0.],
         [    0.],
         [    0.],
         [    0.],
         [    2.]]))

In [5]:
import scipy as sc
q0=pd.concat([i.q0 for i in m.topology.nodes])
assembled = equations_assembler(m.topology)
assembled.Jacobian

array([[ (<bound method spherical.jacobian_i of <constraints.spherical object at 0x000002189F9AAE48>>, array([0, 1, 2, 3, 4, 5, 6]), array([ 7,  8,  9, 10, 11, 12, 13])),
        (<bound method spherical.jacobian_j of <constraints.spherical object at 0x000002189F9AAE48>>, array([0, 1, 2, 3, 4, 5, 6]), array([ 7,  8,  9, 10, 11, 12, 13]))],
       [(<bound method mount.jac of rbs_ground>, array([0, 1, 2, 3, 4, 5, 6])),
        None],
       [None,
        (<bound method rigid.jac of rbs_link>, array([ 7,  8,  9, 10, 11, 12, 13]))]], dtype=object)

In [9]:
q0=pd.concat([i.q0 for i in m.topology.nodes])
jn = jacobian_evaluator(assembled.Jacobian,assembled.Jnzi,assembled.Jf,q0.values)
jn

<11x14 sparse matrix of type '<class 'numpy.float64'>'
	with 29 stored elements in Compressed Sparse Column format>

In [10]:
perm,L,U = sc.linalg.lu(jn.A.T)

In [11]:
perm.shape
(L@U).T.shape
(jn.A@perm).shape

(11, 14)

In [12]:
np.allclose((jn.A@perm)-(L@U).T,np.zeros((11,14)))

True

In [13]:
perm.T@q0.values

array([  6.53281482e-01,  -2.70598050e-01,   6.53281482e-01,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         1.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   1.00000000e+03,   0.00000000e+00,
        -2.70598050e-01,   0.00000000e+00])

In [14]:
m.bodies['rbs_link'].P

array([-0.27059805,  0.65328148, -0.27059805,  0.65328148])

In [27]:
mass_n = sc.sparse.block_diag(assembled['Mf'](assembled['MassMatrix'].diagonal(),q=q0.values)).A
mass_n.shape

(14, 14)

In [28]:
Mi = (perm.T@mass_n@perm)[11:,:]
Mid = Mi[:,0:11]
Mii = Mi[:,11:]
Mii

array([[  1.08856185e+04,   0.00000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   7.26039424e+09,   0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   1.08856185e+04]])

In [38]:
Cf = assembled.Cf
Cv = assembled.Centrifugal
np.concatenate(Cf(Cv,q=q0.values,qi=q0.values))

array([[-0.],
       [-0.],
       [-0.],
       [-0.],
       [-0.],
       [-0.],
       [-0.],
       [-0.],
       [-0.],
       [-0.],
       [-0.],
       [-0.],
       [-0.],
       [-0.]])

In [74]:
extract_ind(jn,q0)

(['rbs_link.z', 'rbs_link.e0', 'rbs_link.y'], array([[ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  1.],
        [ 1.,  0.,  0.],
        [ 0.,  1.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.]]))